In [1]:
# !pip install -q pyspark==3.3.0  spark-nlp==4.3.1
!pip install pyspark
!pip install spark-nlp
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = '/home/ubuntu/spark-3.5.1-bin-hadoop3'

import json
import pickle
import urllib.request
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *

# spark = sparknlp.start(gpu=True)
# Create Spark session with GPU and large memory 24G
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local[*]")\
    .config("spark.driver.memory","84G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp-gpu_2.12:5.3.1")\
    .getOrCreate()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

:: loading settings :: url = jar:file:/home/ubuntu/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-gpu_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-403d82ef-61e8-4669-a91d-0061b84e95bd;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.12;5.3.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 i

Spark NLP version:  5.3.1
Apache Spark version:  3.5.1


In [2]:
# !pip install torch
# !pip install findspark
import findspark
findspark.init()
import torch
print("Pytorch CUDA Available =", torch.cuda.is_available())
print("Pytorch CUDA Device Count =", torch.cuda.device_count())
print("Pytorch CUDA Current Device =", torch.cuda.current_device())
print("Pytorch CUDA Current Device Name =", torch.cuda.get_device_name(torch.cuda.current_device()))

Pytorch CUDA Available = True
Pytorch CUDA Device Count = 4
Pytorch CUDA Current Device = 0
Pytorch CUDA Current Device Name = NVIDIA A10G


In [3]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras # Upgrade to Keras 3.
# from google.colab import drive
# drive.mount('/content/drive/')
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers 
import keras_nlp
import pandas as pd
import numpy as np
import pyspark.sql.functions as F

In [4]:
import os
import tensorflow as tf
from tensorflow.keras.mixed_precision import set_global_policy

# Set TensorFlow to use TensorFlow as backend
os.environ["KERAS_BACKEND"] = "tensorflow"

# Configure the GPU memory and set logical devices before initializing them
physical_devices = tf.config.list_physical_devices("GPU")
if physical_devices:
    try:
        # Assuming you have one GPU available, split it into two logical devices
        tf.config.set_logical_device_configuration(
            physical_devices[0],
            [
                tf.config.LogicalDeviceConfiguration(memory_limit=15360 // 2),
                tf.config.LogicalDeviceConfiguration(memory_limit=15360 // 2),
            ]
        )
    except RuntimeError as e:
        print(e)

# Now initialize the logical devices
logical_devices = tf.config.list_logical_devices("GPU")
print(logical_devices)

base_batch_size = 32
base_learning_rate = 1e-4

# Initialize the distributed training strategy after setting up logical devices
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

scaled_batch_size = base_batch_size * strategy.num_replicas_in_sync 
print(scaled_batch_size)
scaled_learning_rate = base_learning_rate * strategy.num_replicas_in_sync
print(scaled_learning_rate)

strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Set the global policy to mixed_float16
set_global_policy('mixed_float16')

[LogicalDevice(name='/device:GPU:0', device_type='GPU'), LogicalDevice(name='/device:GPU:1', device_type='GPU'), LogicalDevice(name='/device:GPU:2', device_type='GPU'), LogicalDevice(name='/device:GPU:3', device_type='GPU'), LogicalDevice(name='/device:GPU:4', device_type='GPU')]
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


2024-03-26 00:08:46.760161: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')
2024-03-26 00:08:46.760551: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-26 00:08:46.760854: I externa

Number of devices: 5
160
0.0005
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4')


Number of devices: 5


In [5]:
# train_merged_data_conversations_path = 's3a://capstone210/data/train_merged_data_conversations/'
# test_merged_data_conversations_path = 's3a://capstone210/data/test_merged_data_conversations/'

# #latest
# df_train = spark.read.parquet(train_merged_data_conversations_path)
# df_test = spark.read.parquet(test_merged_data_conversations_path)
train_data_path = 's3a://capstone210/data/final/train/'
#test_data_path = 's3a://capstone210/data/final/test/'

df_train = spark.read.parquet(train_data_path)
#df_test = spark.read.parquet(test_data_path)

#latest
df_train = df_train.filter((F.col('label') == 1) | (F.col('label') == 0))
#df_test = df_test.filter((F.col('label') == 1) | (F.col('label') == 0))


24/03/26 00:08:48 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
from pyspark.sql.functions import count

df_train.groupBy('label')\
        .agg(count('*').alias('count'))\
        .show(truncate = False)

+-----+-----+
|label|count|
+-----+-----+
|1    |3119 |
|0    |65992|
+-----+-----+



from the above we have a total of 69,111 total samples and the minority class currently makes up 4.51% of the total population 

lets say we double it? so lets say we want to approx double it to 6000 minority samples. This would then be an increase to 8.68%. We can double it to ~9%

In [7]:
# function to undersample dataset automatically
from pyspark.sql import functions as F

def undersample(df, outcome_col, seed=1234):
    # Split dataset based on outcome
    split0_df = df.filter(F.col(outcome_col) == 0)
    split1_df = df.filter(F.col(outcome_col) == 1)
    # determine which split is major vs minor
    if split0_df.count() > split1_df.count():
        major_df = split0_df
        minor_df = split1_df
    else:
        minor_df = split0_df
        major_df = split1_df
    
    # Calculate current ratio
    current_ratio = major_df.count() / minor_df.count()
    print("Ratio of major vs minor before sampling: {}".format(current_ratio))
    
    # Desired ratio for majority class, adjust this as needed
    desired_ratio = 3
    
    # Calculate the fraction for sampling the majority class
    # Check if current_ratio is already less than or equal to desired_ratio
    if current_ratio > desired_ratio:
        fraction = desired_ratio / current_ratio
    else:
        fraction = 1  # No sampling needed if the current ratio is already less than or equal to desired_ratio

    # Start under-sampling with Spark
    sampled_majority_df = major_df.sample(False, fraction, seed)
    combined_df = sampled_majority_df.unionAll(minor_df)

    # Print final stats
    final_major_count = sampled_majority_df.count()
    final_minor_count = minor_df.count()
    final_ratio = final_major_count / final_minor_count
    print(f"Final major count: {final_major_count}")
    print(f"Final minor count: {final_minor_count}")
    print(f"Final ratio of major vs minor: {final_ratio}")

    return combined_df

# Perform undersampling technique
df_train = undersample(df_train, outcome_col='label')

Ratio of major vs minor before sampling: 21.15806348188522


Final major count: 9493
Final minor count: 3119
Final ratio of major vs minor: 3.04360371914075


In [8]:
df_train.groupBy('label')\
        .agg(count('*').alias('count'))\
        .show(truncate = False)

+-----+-----+
|label|count|
+-----+-----+
|0    |9493 |
|1    |3119 |
+-----+-----+



In [9]:
from xgboost.spark import SparkXGBClassifier

document_assembler = DocumentAssembler()\
  .setInputCol("merged_text")\
  .setOutputCol("document")

tokenizer = Tokenizer().setInputCols(["document"])\
  .setOutputCol("token")

word_embeddings = BertEmbeddings.pretrained('bert_base_cased', 'en')\
  .setInputCols(["document", "token"])\
  .setOutputCol("embeddings") \
  .setDimension(768) \
  .setMaxSentenceLength(512)

nlppipeline = Pipeline().setStages(
  [
    document_assembler,
    tokenizer,
    word_embeddings
  ]
)
fitted_train_df = nlppipeline.fit(df_train).transform(df_train)

bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
[ | ]bert_base_cased download started this may take some time.
Approximate size to download 384.9 MB
Download done! Loading the resource.
[ — ]Using CUDA
[ \ ]

2024-03-26 00:09:30.506693403 [W:onnxruntime:, session_state.cc:1166 VerifyEachNodeIsAssignedToAnEp] Some nodes were not assigned to the preferred execution providers which may or may not have an negative impact on performance. e.g. ORT explicitly assigns shape related ops to CPU to improve perf.
2024-03-26 00:09:30.506725213 [W:onnxruntime:, session_state.cc:1168 VerifyEachNodeIsAssignedToAnEp] Rerunning with verbose output on a non-minimal build will show node assignments.


[OK!]


In [10]:
import pyspark.sql.functions as F
import pyspark.sql.types as T


from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors, VectorUDT

#define a function to get the average embeddings
def avg_vectors(bert_vectors):
  if len(bert_vectors) == 0:
    return [0] * 768
  length = len(bert_vectors)  # This should be the number of vectors, not the dimension of the embeddings.
  num_dimensions = len(bert_vectors[0]["embeddings"])
  avg_vec = [0] * num_dimensions
  for vec in bert_vectors:
    for i, x in enumerate(vec["embeddings"]):
      avg_vec[i] += x
  avg_vec = [x / length for x in avg_vec]  # Correctly average each dimension here.
  return avg_vec

#create a udf
avg_vectors_udf = F.udf(avg_vectors, T.ArrayType(T.DoubleType()))

#Apply the udf
fitted_train_df = fitted_train_df.withColumn("bert_embeddings", avg_vectors_udf(F.col("embeddings")))

def dense_vector(vec):
	return Vectors.dense(vec)

dense_vector_udf = F.udf(dense_vector, VectorUDT())
fitted_train_df = fitted_train_df.withColumn("features", dense_vector_udf(F.col("bert_embeddings")))

In [11]:
pandas_df = fitted_train_df.toPandas()

# SMOTE + TOMEK Links
https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html
https://imbalanced-learn.org/stable/references/generated/imblearn.under_sampling.ClusterCentroids.html

In [ ]:
# pandas_df = fitted_train_df

In [12]:
pandas_df['label'].value_counts()

label
0    9493
1    3119
Name: count, dtype: int64

In [13]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks, ClusterCentroids
from imblearn.pipeline import Pipeline

# Assuming result_df_train is a pandas DataFrame with the 'features' and 'label'
features_array = np.stack(pandas_df['features'].values)
labels_array = pandas_df['label'].values

# Define the desired oversampling ratio for SMOTE
oversampling_ratio = 0.60

# Adjust k_neighbors based on the smallest class size
# For example, if the smallest class has only 3 samples, set k_neighbors to 2 or less
# k_neighbors = min(5, min(pandas_df['label'].value_counts()) - 1)
k_neighbors = 300

# Define the pipeline with both SMOTE and TomekLinks, adjusting k_neighbors in SMOTE
pipeline = Pipeline([
    ('smote', SMOTE(sampling_strategy=oversampling_ratio, k_neighbors=k_neighbors, random_state=42)),
    ('cluster_centroids', ClusterCentroids(sampling_strategy='majority', random_state=42))
    #('tomek', TomekLinks(sampling_strategy='majority'))
])


# Fit and resample the data using the pipeline
X_resampled, y_resampled = pipeline.fit_resample(features_array, labels_array)

# Wrap each numpy array as an object in a pandas Series
features_series = pd.Series([np.array(row) for row in X_resampled], index=range(len(X_resampled)))

# Create a new pandas DataFrame with the single 'features' column and the 'label' column
resampled_data = pd.DataFrame({
    'features': features_series,
    'label': y_resampled
})


/home/ubuntu/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [14]:
resampled_data['label'].value_counts()


label
0    5695
1    5695
Name: count, dtype: int64

In [15]:
resampled_data.shape

(11390, 2)

In [33]:
# path = 's3a://capstone210/data/SMOTE_Cluster_resampling/'

# resampled_data.write.mode('overwrite or append').parquet(path)

In [16]:
!# Assuming `resampled_data` is your Pandas DataFrame
# Specify the file path and name where you want to save the CSV
file_path = '/home/ubuntu/workspace/capstone-210-spring2024/src/notebooks/neural_networks/resampled_training.csv'

# Save the DataFrame to a CSV file
resampled_data.to_csv(file_path, index=False)

In [17]:
resampled_data.head()

,features,label
0,"[0.004133330347637326, 0.1518920719778786, 0.1...",0
1,"[0.4152753662398073, -0.137702038754104, 0.026...",0
2,"[0.06990857089736632, 0.04234974179416895, 0.3...",0
3,"[0.2952515005444487, 0.08149263070275385, 0.41...",0
4,"[0.18737801977179266, -0.0026855284152340823, ...",0
